In [2]:
# Импорты

import math
import numpy as np
import pandas as pd
from datetime import date
import seaborn as sns
import matplotlib.pyplot as plt
import itertools
from time import time 

import holidays
import prophet
from prophet.plot import add_changepoints_to_plot, plot_cross_validation_metrics
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_percentage_error


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
pd.set_option('display.max_columns', 599)
pd.set_option('display.max_rows', 599)

project_id = 'kolesa-android-app'
credentials = 'C:/kolesa-android-app-715adc9bdf26.json'
credentials = service_account.Credentials.from_service_account_file(credentials)

In [ ]:
q = """SELECT * FROM `kolesa-android-app.academy.time_series_dataset` ORDER BY 1"""

In [ ]:
data = pd.read_gbq(query=q, project_id=project_id, credentials=credentials, dialect='standard', progress_bar_type='tqdm') # Запрос данных из GBQ

In [ ]:
#ЕСЛИ У ВАС НЕТ ДОСТУПА К BQ, ТО МОЖНО ЗАГРУЗИТЬ ДАННЫЕ ЧЕРЕЗ CSV

# data = pd.read_csv('Data/data.csv')
# data

In [ ]:
data.sort_values(by='event_date', inplace=True)
data

In [ ]:
data.dtypes

In [ ]:
data['event_date'] = pd.to_datetime(data['event_date'])
data.dtypes

In [ ]:
data.columns = ['ds', 'y']
data.index = pd.to_datetime(data['ds'])
data.head()

In [ ]:
def is_ramadan(ds):
    date = pd.to_datetime(ds)
    return ((date >=pd.to_datetime('2020-04-24')) & (date <= pd.to_datetime('2020-05-24'))
            | (date >=pd.to_datetime('2021-04-13')) & (date <= pd.to_datetime('2021-05-13'))
            | (date >=pd.to_datetime('2022-04-03')) & (date <= pd.to_datetime('2022-05-03')))

data['is_ramadan'] = data['ds'].apply(is_ramadan)
data['ramadan'] = data['is_ramadan'].apply(int)
data.head()
    

In [ ]:
holidays_kz = holidays.KZ(years = (2020, 2021, 2022, 2023))
df_holidays = pd.DataFrame.from_dict(holidays_kz, orient='index').rename(columns={0: 'holiday'})
df_holidays['ds'] = df_holidays.index
df_holidays.head()

In [ ]:
def get_mape(model, actual=data):
    return mean_absolute_percentage_error(actual['y'], model.predict(actual)['yhat'])

def plot(model, actual=data, threshold_changepoints=0.01):
    df_pred = model.predict(actual)
    df_pred.index = df_pred.ds
    df_plot = pd.concat([actual, df_pred], axis=1)
    ax = df_plot[:].plot(y=["y", "yhat"], kind = "line", figsize=(10, 10))
    a = add_changepoints_to_plot(ax, model, df_pred, threshold = threshold_changepoints)

In [ ]:
initial = str(int(data.shape[0]/2)) + ' days'
horizon = str(int(data.shape[0]/20)) + ' days'
period = str(int(data.shape[0]/20)) + ' days'
print('initial:', initial) 
print('horizon:', horizon)
print('period:', period)

In [ ]:
model = Prophet(changepoint_prior_scale=0.9, holidays = df_holidays)

with suppress_stdout_stderr():
    model.fit(data)

    cv_results = cross_validation(
        model, 
        initial=initial, 
        horizon=horizon, 
        period=period,
        parallel="processes"
    )
plot(model)
'MAPE: ', np.average(performance_metrics(cv_results).mape.values)

NameError: name 'Prophet' is not defined